# Kaggle Tabular Playground Series - Aug 2022
https://www.kaggle.com/competitions/tabular-playground-series-aug-2022<br>

In [96]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [97]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import RidgeClassifier

import matplotlib.pyplot as plt

random_seed = 256

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [98]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/train.csv', index_col='id')
train

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,E,158.95,material_7,material_6,6,9,6,16,4,16.301,...,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131,0
26566,E,146.02,material_7,material_6,6,9,10,12,8,17.543,...,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924,0
26567,E,115.62,material_7,material_6,6,9,1,10,1,15.670,...,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364,0


In [99]:
# Check for missing values
train.isna().sum().sum()

20273

In [100]:
# Check more detail for missing values
print('Col name', '\t\t Null values', '\t dtype')
print('=================================================')
for col in train.columns:
        print(col, '\t\t',  train[col].isnull().sum(), '\t\t', train[col].dtype)

Col name 		 Null values 	 dtype
product_code 		 0 		 object
loading 		 250 		 float64
attribute_0 		 0 		 object
attribute_1 		 0 		 object
attribute_2 		 0 		 int64
attribute_3 		 0 		 int64
measurement_0 		 0 		 int64
measurement_1 		 0 		 int64
measurement_2 		 0 		 int64
measurement_3 		 381 		 float64
measurement_4 		 538 		 float64
measurement_5 		 676 		 float64
measurement_6 		 796 		 float64
measurement_7 		 937 		 float64
measurement_8 		 1048 		 float64
measurement_9 		 1227 		 float64
measurement_10 		 1300 		 float64
measurement_11 		 1468 		 float64
measurement_12 		 1601 		 float64
measurement_13 		 1774 		 float64
measurement_14 		 1874 		 float64
measurement_15 		 2009 		 float64
measurement_16 		 2110 		 float64
measurement_17 		 2284 		 float64
failure 		 0 		 int64


In [101]:
# Check for duplicated rows
train.duplicated().sum()

0

In [102]:
# Check categorical data in train
for col in ['product_code', 'attribute_0', 'attribute_1']:
    print(col, ': \t', train[col].unique())

product_code : 	 ['A' 'B' 'C' 'D' 'E']
attribute_0 : 	 ['material_7' 'material_5']
attribute_1 : 	 ['material_8' 'material_5' 'material_6']


In [103]:
# Check categorical data in test
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/test.csv', index_col='id')

for col in ['product_code', 'attribute_0', 'attribute_1']:
    print(col, ': \t', test[col].unique())

product_code : 	 ['F' 'G' 'H' 'I']
attribute_0 : 	 ['material_5' 'material_7']
attribute_1 : 	 ['material_6' 'material_7' 'material_5']


In [104]:
# Get X data
X_train = train.drop(['failure'], axis=1)
X_test = test

# Get features names
feature_names = X_train.columns.values.tolist()

# Categorical encoder. since categories are shared, define one unique encoder
encoder = LabelEncoder()
encoder.fit(['material_5', 'material_6', 'material_7', 'material_8'])
for col in ['attribute_0', 'attribute_1']:
    X_train[col] = encoder.transform(X_train[col])
    X_test[col] = encoder.transform(X_test[col])

# Get y data
y_train = pd.Series(train['failure'])

## Find optimal n_neighbors for KNN imputer

In [105]:
import scipy.stats as stats

features = [f for f in train.columns if f.startswith('measurement') or f=='loading']

for N in range(10):
    temp_X_train = X_train.copy()
    temp_y_train = y_train.copy()
    
    temp_imp = KNNImputer(missing_values=np.nan, n_neighbors = N+1)
        
    # Impute data for train and test
    for code in X_train.product_code.unique():
        temp_X_train.loc[temp_X_train.product_code==code, features] = temp_imp.fit_transform(temp_X_train.loc[temp_X_train.product_code==code, features])
    
    avg_corr = 0
    # Calculate average point-biserial correlation
    for i in range(len(features)):
        feature = features[i]
        binary = temp_y_train
        numerical = temp_X_train[feature]
        corr = stats.pointbiserialr(binary, numerical).correlation
        pvalue = stats.pointbiserialr(binary, numerical).pvalue
        avg_corr = avg_corr + abs(corr)
        
    avg_corr = avg_corr / len(features)
    print(N+1, 'Neighbors:', '\t\t', avg_corr)

1 Neighbors: 		 0.015922776072366262
2 Neighbors: 		 0.01589440824436477
3 Neighbors: 		 0.016117318138899238
4 Neighbors: 		 0.016118265136516086
5 Neighbors: 		 0.016115403501276503
6 Neighbors: 		 0.01619439586031882
7 Neighbors: 		 0.0161015728938419
8 Neighbors: 		 0.016085719020423602
9 Neighbors: 		 0.016029947139380725
10 Neighbors: 		 0.01605852691857642


## Missing values imputation

In [106]:
# Imputing null values
# Create our imputer to replace missing values
#imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = KNNImputer(missing_values=np.nan, n_neighbors=6)

# Impute data for train and test
features = [f for f in train.columns if f.startswith('measurement') or f=='loading']

for code in X_train.product_code.unique():
    X_train.loc[X_train.product_code==code, features] = imp.fit_transform(X_train.loc[X_train.product_code==code, features])
    
for code in X_test.product_code.unique():
    X_test.loc[X_test.product_code==code, features] = imp.fit_transform(X_test.loc[X_test.product_code==code, features])

# Since product_code is different between train and test, will be dropped
X_train = X_train.drop(['product_code'], axis=1)
X_test = X_test.drop(['product_code'], axis=1)

## Model creation
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html<br>

In [107]:
# By default model
params = {#'base_estimator' : None, 
          #'n_estimators' : 50, 
          #'learning_rate' : 1.0, 
          #'algorithm' : 'SAMME.R'
         }

model = AdaBoostClassifier(**params,
                           random_state=random_seed,
                           )

## Cross Validation
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html<br>
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter<br>

In [108]:
cv = StratifiedKFold(n_splits=5)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
#scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)

scores.mean()

#0.5780383200493329

0.5780383200493329

In [109]:
model.get_params()

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': 256}

## Optuna Optimization

In [110]:
import optuna

from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [111]:
def objective(trial):
       
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1.0, step=1e-4)
    algorithm = trial.suggest_categorical('algorithm',['SAMME', 'SAMME.R'])
        
    params = {'n_estimators': n_estimators,
              'learning_rate': learning_rate,
              'algorithm': algorithm,
             }
    
    model = AdaBoostClassifier(**params,
                               random_state=random_seed,
                               )
    
    cv = StratifiedKFold(n_splits=5)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
    #scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
        
    return scores.mean()

In [112]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

np.seterr(all='ignore')

# Create Optuna Trial
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler(seed=random_seed))

# Run trials
study.optimize(objective, n_trials = 200)
#study.optimize(objective, timeout = int(3600*1))    # an hour * X

[I 2022-08-16 02:52:49,584] A new study created in memory with name: no-name-f5f831d8-b7ab-4fa9-9b44-9c7e2745a0c3
[I 2022-08-16 02:52:51,911] Trial 0 finished with value: 0.5794384564107353 and parameters: {'n_estimators': 18, 'learning_rate': 0.5862, 'algorithm': 'SAMME'}. Best is trial 0 with value: 0.5794384564107353.
[I 2022-08-16 02:52:53,785] Trial 1 finished with value: 0.5847875809532568 and parameters: {'n_estimators': 14, 'learning_rate': 0.403, 'algorithm': 'SAMME'}. Best is trial 1 with value: 0.5847875809532568.
[I 2022-08-16 02:53:14,960] Trial 2 finished with value: 0.5850220029175233 and parameters: {'n_estimators': 163, 'learning_rate': 0.3797, 'algorithm': 'SAMME.R'}. Best is trial 2 with value: 0.5850220029175233.
[I 2022-08-16 02:53:28,552] Trial 3 finished with value: 0.5883484859585424 and parameters: {'n_estimators': 113, 'learning_rate': 0.5232, 'algorithm': 'SAMME'}. Best is trial 3 with value: 0.5883484859585424.
[I 2022-08-16 02:53:39,754] Trial 4 finished wi

In [113]:
# Best trial
print('Best trial score:', study.best_trial.value)
study.best_trial.params

Best trial score: 0.5945346408264982


{'n_estimators': 32,
 'learning_rate': 0.15009999999999998,
 'algorithm': 'SAMME.R'}

## Visualization for Hyperparameter Optimization Analysis

In [114]:
plot_optimization_history(study)

In [115]:
plot_param_importances(study)

In [116]:
plot_slice(study)

## Get Best Model

In [117]:
# Create model with best trial parameters
params = {'n_estimators': study.best_trial.params['n_estimators'],
          'learning_rate': study.best_trial.params['learning_rate'],
          'algorithm': study.best_trial.params['algorithm'],
         }

best_model = AdaBoostClassifier(**params,
                                random_state=random_seed,
                                )

In [118]:
best_model.get_params()

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 0.15009999999999998,
 'n_estimators': 32,
 'random_state': 256}

## Train Model

In [119]:
# Train best model with all train data
best_model.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=0.15009999999999998, n_estimators=32,
                   random_state=256)

## Submission

In [120]:
prediction = best_model.predict(X_test).squeeze()
probabilities = best_model.predict_proba(X_test)[:,1].squeeze()

row_id =  test.index.values
submission = pd.DataFrame({'id' : row_id, 'failure' : probabilities})

submission.to_csv('submission.csv', index=False)

submission.head()

,id,failure
0,26570,0.425527
1,26571,0.424448
2,26572,0.423338
3,26573,0.423338
4,26574,0.454796
